<a href="https://colab.research.google.com/github/brettdavies/UCSD-ML-AI-Projects/blob/main/Student_MLE_MiniProject_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Transfer Learning with Keras

Transfer learning is a machine learning technique where a model trained on one task is used as a starting point to solve a different but related task. Instead of training a model from scratch, transfer learning leverages the knowledge learned from the source task and applies it to the target task. This approach is especially useful when the target task has limited data or computational resources.

In transfer learning, the pre-trained model, also known as the "base model" or "source model," is typically trained on a large dataset and a more general problem (e.g., image classification on ImageNet, a vast dataset with millions of labeled images). The knowledge learned by the base model in the form of feature representations and weights captures common patterns and features in the data.

To perform transfer learning, the following steps are commonly followed:

1. Pre-training: The base model is trained on a source task using a large dataset, which can take a considerable amount of time and computational resources.

2. Feature Extraction: After pre-training, the base model is used as a feature extractor. The last few layers (classifier layers) of the model are discarded, and the remaining layers (feature extraction layers) are retained. These layers serve as feature extractors, producing meaningful representations of the data.

3. Fine-tuning: The feature extraction layers and sometimes some of the earlier layers are connected to a new set of layers, often called the "classifier layers" or "task-specific layers." These layers are randomly initialized, and the model is trained on the target task with a smaller dataset. The weights of the base model can be frozen during fine-tuning, or they can be allowed to be updated with a lower learning rate to fine-tune the model for the target task.

Transfer learning has several benefits:

1. Reduced training time and resource requirements: Since the base model has already learned generic features, transfer learning can save time and resources compared to training a model from scratch.

2. Improved generalization: Transfer learning helps the model generalize better to the target task, especially when the target dataset is small and dissimilar from the source dataset.

3. Better performance: By starting from a model that is already trained on a large dataset, transfer learning can lead to better performance on the target task, especially in scenarios with limited data.

4. Effective feature extraction: The feature extraction layers of the pre-trained model can serve as powerful feature extractors for different tasks, even when the task domains differ.

Transfer learning is commonly used in various domains, including computer vision, natural language processing (NLP), and speech recognition, where pre-trained models are fine-tuned for specific applications like object detection, sentiment analysis, or speech-to-text.

In this mini-project you will perform fine-tuning using Keras with a pre-trained VGG16 model on the CIFAR-10 dataset.

First, import all the libraries you'll need.

In [114]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import Input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision and machine learning. It stands for the "Canadian Institute for Advanced Research 10" dataset. CIFAR-10 was created by researchers at the CIFAR institute and was originally introduced as part of the Neural Information Processing Systems (NIPS) 2009 competition.

The dataset consists of 60,000 color images, each of size 32x32 pixels, belonging to ten different classes. Each class contains 6,000 images. The ten classes in CIFAR-10 are:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

The images are evenly distributed across the classes, making CIFAR-10 a balanced dataset. The dataset is divided into two sets: a training set and a test set. The training set contains 50,000 images, while the test set contains the remaining 10,000 images.

CIFAR-10 is often used for tasks such as image classification, object recognition, and transfer learning experiments. The relatively small size of the images and the variety of classes make it a challenging dataset for training machine learning models, especially deep neural networks. It also serves as a good dataset for teaching and learning purposes due to its manageable size and straightforward class labels.

Here are your tasks:

1. Load the CIFAR-10 dataset after referencing the documentation [here](https://keras.io/api/datasets/cifar10/).
2. Normalize the pixel values so they're all in the range [0, 1].
3. Apply One Hot Encoding to the train and test labels using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
4. Further split the the training data into training and validation sets using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use only 10% of the data for validation.  

In [115]:
# Load the CIFAR-10 dataset
(images_train, labels_train), (images_test, labels_test) = cifar10.load_data()

In [116]:
# Normalize the pixel values to [0, 1]
images_train, images_test = images_train.astype(dtype=np.float32) / 255.0, images_test.astype(dtype=np.float32) / 255.0

In [117]:
# One-hot encode the labels
labels_train = to_categorical(labels_train, dtype = "float32")
labels_test = to_categorical(labels_test, dtype = "float32")

In [118]:
# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(images_train, labels_train, test_size=0.15, random_state=42)

assert X_train.shape == (42500, 32, 32, 3)
assert X_test.shape == (7500, 32, 32, 3)
assert y_train.shape == (42500, 10)
assert y_test.shape == (7500, 10)

VGG16 (Visual Geometry Group 16) is a deep convolutional neural network architecture that was developed by the Visual Geometry Group at the University of Oxford. It was proposed by researchers Karen Simonyan and Andrew Zisserman in their paper titled "Very Deep Convolutional Networks for Large-Scale Image Recognition," which was presented at the International Conference on Learning Representations (ICLR) in 2015.

The VGG16 architecture gained significant popularity for its simplicity and effectiveness in image classification tasks. It was one of the pioneering models that demonstrated the power of deeper neural networks for visual recognition tasks.

Key characteristics of the VGG16 architecture:

1. Architecture: VGG16 consists of a total of 16 layers, hence the name "16." These layers are stacked one after another, forming a deep neural network.

2. Convolutional Layers: The main building blocks of VGG16 are the convolutional layers. It primarily uses 3x3 convolutional filters throughout the network, which allows it to capture local features effectively.

3. Max Pooling: After each set of convolutional layers, VGG16 applies max-pooling layers with 2x2 filters and stride 2, which halves the spatial dimensions (width and height) of the feature maps and reduces the number of parameters.

4. Fully Connected Layers: Towards the end of the network, VGG16 has fully connected layers that act as a classifier to make predictions based on the learned features.

5. Activation Function: The network uses the Rectified Linear Unit (ReLU) activation function for all hidden layers, which helps with faster convergence during training.

6. Number of Filters: The number of filters in each convolutional layer is relatively small compared to more recent architectures like ResNet or InceptionNet. However, stacking multiple layers allows VGG16 to learn complex hierarchical features.

7. Output Layer: The output layer consists of 1000 units, corresponding to 1000 ImageNet classes. VGG16 was originally trained on the large-scale ImageNet dataset, which contains millions of images from 1000 different classes.

VGG16 was instrumental in showing that increasing the depth of a neural network can significantly improve its performance on image recognition tasks. However, the main drawback of VGG16 is its high number of parameters, making it computationally expensive and memory-intensive to train. Despite this limitation, VGG16 remains an essential benchmark architecture and has paved the way for even deeper and more efficient models in the field of computer vision, such as ResNet, DenseNet, and EfficientNet.

Here are your tasks:

1. Load [VGG16](https://keras.io/api/applications/vgg/#vgg16-function) as a base model. Make sure to exclude the top layer.
2. Freeze all the layers in the base model. We'll be using these weights as a feature extraction layer to forward to layers that are trainable.

In [119]:
# Load the pre-trained VGG16 model (excluding the top classifier)
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

In [120]:
# Freeze the layers in the base model
base_model.trainable = False

Now, we'll add some trainable layers to the base model.

1. Using the base model, add a [GlobalAveragePooling2D](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) layer, followed by a [Dense](https://keras.io/api/layers/core_layers/dense/) layer of length 256 with ReLU activation. Finally, add a classification layer with 10 units, corresponding to the 10 CIFAR-10 classes, with softmax activation.
2. Create a Keras [Model](https://keras.io/api/models/model/) that takes in approproate inputs and outputs.

In [121]:
# Add a global average pooling layer
inputs = Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)

In [122]:
# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)

In [123]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
outputs = Dense(10, activation='softmax')(x)

In [124]:
# Create the fine-tuned model
model = Model(inputs, outputs)
model.summary(show_trainable=True)

Model: "model_8"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_12 (InputLayer)       [(None, 32, 32, 3)]       0         Y          
                                                                            
 vgg16 (Functional)          (None, 1, 1, 512)         1471468   N          
                                                       8                    
                                                                            
 global_average_pooling2d_5  (None, 512)               0         Y          
  (GlobalAveragePooling2D)                                                  
                                                                            
 dense_10 (Dense)            (None, 256)               131328    Y          
                                                                            
 dense_11 (Dense)            (None, 10)                2570

With your model complete it's time to train it and assess its performance.

1. Compile your model using an appropriate loss function. Feel free to play around with the optimizer, but a good starting optimizer might be Adam with a learning rate of 0.001.
2. Fit your model on the training data. Use the validation data to print the accuracy for each epoch. Try training for 10 epochs. Note, training can take a few hours so go ahead and grab a cup of coffee.

**Optional**: See if you can implement an [Early Stopping](https://keras.io/api/callbacks/early_stopping/) criteria as a callback function.

In [125]:
# Compile the model
model.compile(
    optimizer = keras.optimizers.Adam(lr=0.01),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [126]:
epochs = 30
callback = keras.callbacks.EarlyStopping(
      monitor = 'val_accuracy',
      patience = 3
)

In [127]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback], verbose=2)

Epoch 1/30
1329/1329 - 6s - loss: 1.3694 - accuracy: 0.5241 - val_loss: 1.2308 - val_accuracy: 0.5665 - 6s/epoch - 5ms/step
Epoch 2/30
1329/1329 - 5s - loss: 1.1818 - accuracy: 0.5883 - val_loss: 1.1905 - val_accuracy: 0.5811 - 5s/epoch - 4ms/step
Epoch 3/30
1329/1329 - 5s - loss: 1.1160 - accuracy: 0.6119 - val_loss: 1.1388 - val_accuracy: 0.6000 - 5s/epoch - 4ms/step
Epoch 4/30
1329/1329 - 5s - loss: 1.0676 - accuracy: 0.6265 - val_loss: 1.1405 - val_accuracy: 0.6005 - 5s/epoch - 4ms/step
Epoch 5/30
1329/1329 - 5s - loss: 1.0224 - accuracy: 0.6420 - val_loss: 1.1283 - val_accuracy: 0.6097 - 5s/epoch - 4ms/step
Epoch 6/30
1329/1329 - 5s - loss: 0.9836 - accuracy: 0.6570 - val_loss: 1.1406 - val_accuracy: 0.6085 - 5s/epoch - 4ms/step
Epoch 7/30
1329/1329 - 5s - loss: 0.9469 - accuracy: 0.6680 - val_loss: 1.0930 - val_accuracy: 0.6251 - 5s/epoch - 4ms/step
Epoch 8/30
1329/1329 - 5s - loss: 0.9110 - accuracy: 0.6790 - val_loss: 1.1229 - val_accuracy: 0.6123 - 5s/epoch - 4ms/step
Epoch 9/

With your model trained, it's time to assess how well it performs on the test data.

1. Use your trained model to calculate the accuracy on the test set. Is the model performance better than random?
2. Experiment! See if you can tweak your model to improve performance.  

In [128]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(images_test, labels_test, verbose=2)
print(f"Test Accuracy: {test_acc:.2%}")

313/313 - 1s - loss: 1.1418 - accuracy: 0.6141 - 946ms/epoch - 3ms/step
Test Accuracy: 61.41%
